In [1]:
import yfinance as yf

token = 'BTC-USD'
# start_date = '2015-01-01'

data = yf.download(token, interval='1d')
data.to_csv(f'{token}.csv')
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2024-12-18,106147.296875,106470.609375,100041.539062,100041.539062,100041.539062,93865656139
2024-12-19,100070.687500,102748.148438,95587.679688,97490.953125,97490.953125,97221662392
2024-12-20,97484.695312,98098.914062,92175.179688,97755.929688,97755.929688,105634083408


In [2]:
import pandas as pd 

data = pd.read_csv(f'{token}.csv', parse_dates=['Date'], index_col=['Date'])
data.drop(['Adj Close', 'Open', 'High', 'Low', 'Volume'], axis=1, inplace=True)
data

,Close
Date,
2014-09-17,457.334015
2014-09-18,424.440002
2014-09-19,394.795990
2014-09-20,408.903992
2014-09-21,398.821014
...,...
2024-12-13,101459.257812
2024-12-14,101372.968750
2024-12-15,104298.695312


In [7]:
import pandas as pd 
import ta

data['sma'] = ta.trend.sma_indicator(data['Close'], window=30)
data['ema20'] = ta.trend.ema_indicator(data['Close'], window=20)
data['ema50'] = ta.trend.ema_indicator(data['Close'], window=50)
data['ema200'] = ta.trend.ema_indicator(data['Close'], window=200)


# Añadir RSI
data['rsi'] = ta.momentum.rsi(data['Close'], window=14)

# Añadir MACD
macd = ta.trend.MACD(data['Close'], window_slow=26, window_fast=12, window_sign=9)
data['macd'] = macd.macd()
data['macd_signal'] = macd.macd_signal()

#Precio Futuro
data['Close_future'] = data['Close'].shift(-1)
# Dirección del Movimiento de Precio
data['Price_Up'] = (data['Close_future'] > data['Close']).astype(int)


#Boramos datos nulos
data.dropna(inplace=True)

data.to_csv('BTC-Indicators.csv',index=['Date'])
data = pd.read_csv('BTC-Indicators.csv', parse_dates=['Date'], index_col=['Date'])
data

,Open,High,Low,Close,Adj Close,Volume,sma,ema20,ema50,ema200,rsi,macd,macd_signal,Close_future,Price_Up
Date,,,,,,,,,,,,,,,
2015-10-20,263.571991,270.834991,263.226990,269.463013,269.463013,30889800,244.176467,252.197868,245.967634,247.938681,71.450604,7.668835,5.523274,266.272003,0
2015-10-21,269.306000,270.769989,263.838989,266.272003,266.272003,25637300,245.482701,253.538262,246.763884,248.121102,66.900158,7.813398,5.981299,274.023010,1
2015-10-22,266.496002,276.510010,266.135010,274.023010,274.023010,37808600,246.929534,255.489190,247.832869,248.378833,71.626972,8.455932,6.476225,276.496002,1
2015-10-23,273.648987,278.683990,273.542999,276.496002,276.496002,29442500,248.469968,257.489839,248.956913,248.658605,72.954043,9.060253,6.993031,281.653992,1
2015-10-24,276.503998,281.705994,276.503998,281.653992,281.653992,25942400,250.040800,259.791187,250.239152,248.986918,75.525315,9.841937,7.562812,283.679993,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-16,104293.578125,107780.578125,103322.984375,106029.718750,106029.718750,91020417816,97501.756771,98891.717554,90677.438970,72849.742753,69.192878,3707.914747,3675.381559,106140.601562,1
2024-12-17,106030.687500,108268.445312,105291.734375,106140.601562,106140.601562,68589364868,98044.915104,99582.087460,91283.837503,73180.995080,69.336619,3847.987439,3709.902735,100041.539062,0
2024-12-18,106147.296875,106470.609375,100041.539062,100041.539062,100041.539062,93865656139,98361.545052,99625.844755,91627.276780,73448.264174,54.322688,3427.344393,3653.391067,97490.953125,0


In [6]:
features = ['Close', 'sma', 'ema20', 'ema50', 'ema200', 'rsi', 'macd', 'macd_signal']
target = ['Price_Up']
# Separar X e y
X = data[features]
y = data[target]


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, shuffle= False)


ValueError: With n_samples=0, test_size=0.01 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
